# I. Importation des bibliothèques

In [ ]:
%%capture
%pip install --quiet fuzzywuzzy python-Levenshtein excel-formulas-calculator
%pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
%%capture
import warnings

warnings.filterwarnings("ignore")
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
import math

import openpyxl as pyxl
from openhexa.sdk import workspace

try:
    from efc.interfaces.iopenpyxl import OpenpyxlInterface
except ImportError or ModuleNotFoundError:
    %pip install --quiet excel-formulas-calculator
    from efc.interfaces.iopenpyxl import OpenpyxlInterface

## 1. Rafraîchissement des bibliothèques

In [3]:
%%capture
# Ajout du chemin d'accès pour l'importation des bibliothèques
os.chdir(Path(workspace.files_path, "Fichier Suivi de Stock/code/pipelines"))

from importlib import reload

import compute_indicators
import export_file_to_google_drive as ggdrive
import generate_stock_tracking_file as gstf

# Importation des réquêtes sql
from compute_indicators.queries import QUERY_ETAT_STOCK_PROGRAMME
from database_operations import process_statut_prod, stock_sync_manager

# Reload modules
reload(compute_indicators)
reload(stock_sync_manager)
reload(gstf)
reload(ggdrive)

# II. Définition des paramètres

## 1. Variables requises pour l'actualisation du fichier de suivi de stock

1. **Mois de création du rapport** : Fournie en tant qu'entrée du pipeline.

2. **Programme** : Le programme concerné pour lequel le fichier de suivi de stock.

3. **Fichier Suivi de Stock** : Le fichier suivi de stock finalisé après la réunion mensuelle.

In [ ]:
date_report, programme, fp_suvi_stock = (
    "Janvier",
    "PNLP",
    "Fichier Suivi de Stock/code/pipelines/generate_stock_tracking_file/Template Fichier Suivi de Stock/Fichier Suivi de Stock Template.xlsx",
    # "Fichier Suivi de Stock PNLS-JUILLET-2024.xlsx",
)

In [ ]:
month_export, date_report = date_report, compute_indicators.utils.format_date(date_report)

## 2. Test pour s'assurer qu'il y a bien des données déjà présentes dans la base de données pour ce programme en question

L'idée est de vérifier au préalable, avant d'apporter des modifications au programme en question, que des données existent déjà dans la base de données.

In [ ]:
stock_sync_manager.initialize_database_connection()

schema_name = "suivi_stock"

In [ ]:
df_ = stock_sync_manager.get_table_data(
    query=f"""
    select * 
    from {schema_name}.stock_track st 
    inner join {schema_name}.dim_produit_stock_track prod ON st.id_dim_produit_stock_track_fk = prod.id_dim_produit_stock_track_pk
    where prod.programme='{programme}' and date_report='{date_report}'
    limit 2
    """
)

assert df_.shape[0] != 0, (
    f"Le mois séléecitonné {date_report} n'a pas de données présente dans la base de données"
)

del df_

# III.📥Importation des Données
  
L'utilisateur doit veiller à ce que les fichiers respectent le format attendu et soient placés dans les répertoires dédiés avant de procéder au traitement.

## 📌1. Importation du fichier `Fichier Suivi des Stocks`

- **Emplacement du fichier :** Le fichier doit être placé dans le répertoire dédié :  
  **`Fichier Suivi de Stock/data/<programme>/Fichier Suivi de Stock`**
  
- **En cas d'erreur :**  
  - Vérifiez que le fichier est bien présent dans le répertoire.  
  - Assurez-vous que toutes les colonnes requises sont bien renseignées.  
  - Contrôlez que le fichier est bien accessible et non corrompu.

In [ ]:
fp_suvi_stock = (
    Path(workspace.files_path)
    / f"Fichier Suivi de Stock/data/{programme}/Fichier Suivi de Stock"
    / Path(fp_suvi_stock).name
)
src_wb = pyxl.load_workbook(fp_suvi_stock)
sheetnames = src_wb.sheetnames

## 📌2. Importation de la feuille `Etat de stock de la NPSP`

In [ ]:
sheet_stock_npsp = compute_indicators.utils.check_if_sheet_name_in_file("Etat de stock", sheetnames)

assert sheet_stock_npsp is not None, print(
    f"La feuille `Etat de stock` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_etat_stock_npsp = pd.read_excel(fp_suvi_stock, sheet_name=sheet_stock_npsp, skiprows=4)

del sheet_stock_npsp

df_etat_stock_npsp = df_etat_stock_npsp.loc[df_etat_stock_npsp["Nouveau code"].notna()]

df_etat_stock_npsp = compute_indicators.file_utils.process_etat_stock_npsp(
    df_etat_stock_npsp, date_report, programme
)

df_etat_stock_npsp.head(2)

## 📌3. Importation de la feuille `Stock detaille`

In [ ]:
sheet_stock_detaille = compute_indicators.utils.check_if_sheet_name_in_file(
    "Stock detaille", sheetnames
)

assert sheet_stock_detaille is not None, print(
    f"La feuille `Stock detaille` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_stock_detaille = pd.read_excel(fp_suvi_stock, sheet_name=sheet_stock_detaille)

max_date_year = pd.Timestamp.max.year

try:
    df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
        "Date limite de consommation"
    ].apply(lambda x: x if x.year < max_date_year else x.replace(year=max_date_year - 1))
except Exception:
    df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
        "Date limite de consommation"
    ].str.strip()

    df_stock_detaille["Date limite de consommation"] = df_stock_detaille[
        "Date limite de consommation"
    ].apply(
        lambda x: x if int(x[-4:]) < max_date_year else x.replace(x[-4:], str(max_date_year - 1))
    )

df_stock_detaille["Date limite de consommation"] = pd.to_datetime(
    df_stock_detaille["Date limite de consommation"], format="%d/%m/%Y"
)

del sheet_stock_detaille, max_date_year

df_stock_detaille.head(2)

## 📌4. Importation de la feuille `Distribution X3`

In [ ]:
sheet_distribution_x3 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Distribution X3", sheetnames
)

assert sheet_distribution_x3 is not None, print(
    f"La feuille `Distribution X3` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_distribution = pd.read_excel(fp_suvi_stock, sheet_name=sheet_distribution_x3)

del sheet_distribution_x3

df_distribution.head(2)

## 📌5. Importation de la feuille `Receptions`

In [ ]:
sheet_reception = compute_indicators.utils.check_if_sheet_name_in_file("Receptions", sheetnames)

assert sheet_reception is not None, print(
    f"La feuille `Receptions` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_receptions = pd.read_excel(fp_suvi_stock, sheet_name=sheet_reception)

del sheet_reception

df_receptions.head(2)

## 📌6. Importation de la feuille `PPI`

In [ ]:
sheet_ppi = compute_indicators.utils.check_if_sheet_name_in_file("PPI", sheetnames)

assert sheet_ppi is not None, print(
    f"La feuille `PPI` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_ppi = pd.read_excel(fp_suvi_stock, sheet_name=sheet_ppi, skiprows=2)

del sheet_ppi

df_ppi.head(2)

## 📌7. Importation de la feuille  `Prélèvement`

In [ ]:
sheet_prelev = compute_indicators.utils.check_if_sheet_name_in_file("Prelèvement CQ", sheetnames)

assert sheet_prelev is not None, print(
    f"La feuille `Prelèvement CQ` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_prelevement = pd.read_excel(fp_suvi_stock, sheet_name=sheet_prelev, skiprows=2)

del sheet_prelev

df_prelevement.head(2)

## 📌8. Importation de la feuille `Plan d'approvisionnement`

In [ ]:
sheet_approv = compute_indicators.utils.check_if_sheet_name_in_file("Plan d'appro", sheetnames)

assert sheet_approv is not None, print(
    f"La feuille `Plan d'appro` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_plan_approv = pd.read_excel(fp_suvi_stock, sheet_name=sheet_approv)

df_plan_approv.head()

## 📌9. Importation de la feuille `Statut Produits`

In [ ]:
sheet_statut_prod = compute_indicators.utils.check_if_sheet_name_in_file(
    "Statut Produits", sheetnames
)

assert sheet_statut_prod is not None, print(
    f"La feuille `Statut Produits` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_statut_prod = pd.read_excel(fp_suvi_stock, sheet_name=sheet_statut_prod, skiprows=1)
df_statut_prod["programme"] = programme
df_statut_prod.head()

## 📌10. Importation de la feuille `Annexe 1 - Consolidation`

In [ ]:
sheet_annexe_1 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Annexe 1 - Consolidation", sheetnames
)

assert sheet_annexe_1 is not None, print(
    f"La feuille `Annexe 1 - Consolidation` n'est pas dans la liste {sheetnames} du classeur excel"
)

df_annexe = pd.read_excel(
    fp_suvi_stock, sheet_name=sheet_annexe_1, skiprows=2, usecols="A:T"
).dropna(how="all")

COLUMN_MAPPING = {
    "Stock Théorique": "stock_theorique_mois_precedent",
}
df_annexe.rename(
    columns={"CODE": "code_produit"},
    inplace=True,
)

df_annexe.rename(
    columns=lambda col: next(
        (v for k, v in COLUMN_MAPPING.items() if re.search(k, col, re.I)), col
    ),
    inplace=True,
)

df_annexe.head(3)

# IV. Mise à jour des informations

## 1. Mise à jour des informations sur le produit en se basant sur la feuille `Statut Produits`

À l’issue des réunions mensuelles, des ajustements peuvent être effectués sur les informations de produits.
Ces ajustements incluent :

 * l’ajout de nouveaux produits,
 * la suppression de produits existants,
 * la modification des attributs ou métadonnées associées à certains produits.

La fonction suivante a pour objectif de journaliser ces opérations afin d'assurer une traçabilité complète des modifications apportées.

In [ ]:
process_statut_prod(df_statut_prod, schema_name, stock_sync_manager)

## 2. Mise à jour des informations Produits en se basant sur la feuille `Plan d'appro`

In [ ]:
df_pa = df_plan_approv.rename(
    columns={
        "Acronym": "acronym",
        "Coût unitaire moyen (en dollar)": "cout_unitaire_moyen_qat",
        "Facteur de conversion de QAT vers SAGE": "facteur_de_conversion_qat_sage",
    }
)

In [ ]:
stock_sync_manager.synchronize_product_metadata(df_pa, programme)

del df_pa

## 2. Recherche des modifications sur `Annexe 1 - Consolidation`

In [ ]:
interface = OpenpyxlInterface(wb=src_wb, use_cache=True)
interface.clear_cache()
data_list = []
for row in src_wb[sheet_annexe_1].iter_rows(min_row=5, max_col=19):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)

In [ ]:
if data_list:
    df_annexe = pd.DataFrame(data_list, columns=df_annexe.columns)

    df_annexe.fillna(np.nan, inplace=True)

    del data_list, data, result

df_annexe.head(3)

## 3. Recherche des modifications sur les `DMM`

In [ ]:
# DMM échélonnée par mois
interface.clear_cache()
data_list = []
for start, row in enumerate(
    src_wb[sheet_annexe_1].iter_rows(
        min_row=4,
        min_col=pyxl.utils.column_index_from_string("V"),
        max_col=pyxl.utils.column_index_from_string("BE"),
    ),
    start=1,
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    if start == 1:
        new_data = []
        unnamed_counter = 22  # Colonne de début des DMM
        for value in data:
            if value is None:
                new_data.append(f"Unnamed: {unnamed_counter}")
                unnamed_counter += 23
            else:
                new_data.append(value)
        data = new_data
    data_list.append(data)
del new_data

In [ ]:
df_dmm = pd.concat(
    [df_annexe[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
)

In [ ]:
df_stock_track_dmm = df_dmm[[col for col in df_dmm.columns if "Unnamed" not in str(col)]]

df_stock_track_dmm = pd.melt(
    df_stock_track_dmm, id_vars="code_produit", var_name="date_report", value_name="dmm"
)  # .drop_duplicates()

df_stock_track_dmm["date_report"] = (
    df_stock_track_dmm["date_report"]
    .apply(lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d"))
    .astype("<M8[ns]")
)

In [ ]:
# Extraction des informations pour le mois courant
interface.clear_cache()
data_list = []
for row in src_wb[sheet_annexe_1].iter_rows(
    min_row=3,
    min_col=pyxl.utils.column_index_from_string("BG"),
    max_col=pyxl.utils.column_index_from_string("BJ"),
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)

In [ ]:
df_dmm_curent_month = pd.concat(
    [df_annexe[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
).dropna(how="all")

df_dmm_curent_month["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")
df_dmm_curent_month.columns = df_dmm_curent_month.columns.str.replace("\n", " ")
df_dmm_curent_month.rename(
    columns={
        "Nbre de mois de considérés": "nbre_mois_consideres",
        "Distributions enregistrées sur les mois de considérés": "distributions_mois_consideres",
        "DMM Calculée  (à valider pour ce mois)": "dmm_calculee",
        "COMMENTAIRE": "commentaire",
    },
    inplace=True,
)

df_dmm_curent_month.head(2)

In [ ]:
assert (
    df_stock_track_dmm.merge(
        df_dmm_curent_month, how="left", on=["code_produit", "date_report"]
    ).shape[0]
    == df_stock_track_dmm.shape[0]
)

df_stock_track_dmm = df_stock_track_dmm.merge(
    df_dmm_curent_month, how="left", on=["code_produit", "date_report"]
)

In [ ]:
cols_df_dmm = df_dmm.columns.to_list()

mapping = {
    col: cols_df_dmm[i - 1] for i, col in enumerate(cols_df_dmm) if "Unnamed" in str(col) and i > 0
}


df = pd.melt(
    df_dmm,
    id_vars=["code_produit"],
    value_vars=[col for col in df_dmm.columns if not pd.isna(col)],
    var_name="date_report",
)

df["date_report"] = df["date_report"].map(mapping)

df = df.loc[df.value.notna()]

df_stock_track_dmm_histo = (
    df_stock_track_dmm[["code_produit", "date_report", "dmm"]]
    .merge(df.drop(columns="value"), on=["code_produit", "date_report"])
    .rename(columns={"date_report": "date_report_prev"})
)

df_stock_track_dmm_histo["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")

del df, mapping

df_stock_track_dmm_histo.head()

## 4. Recherche des modifications sur les `CMM`

In [ ]:
eomonth = (pd.to_datetime(date_report) + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")

df_stock_prog_nat = stock_sync_manager.get_table_data(
    query=QUERY_ETAT_STOCK_PROGRAMME.format(eomonth=eomonth, programme=programme)
)

df_stock_prog_nat["Code_produit"] = df_stock_prog_nat["Code_produit"].astype(int)

df_stock_prog_nat.head(3)

In [ ]:
header_row = list(
    src_wb[sheet_annexe_1].iter_rows(
        min_row=4,
        max_row=4,
        min_col=pyxl.utils.column_index_from_string("BL"),
        max_col=pyxl.utils.column_index_from_string("CJ"),
    )
)[0]

dico_cols = {}
for cell in header_row:
    if not isinstance(cell, pyxl.cell.MergedCell):
        dico_cols[str(cell.value)[:10]] = cell.column

In [ ]:
# Mise à jour des valeur de cellule car la formule ArrondiSup n'est pas prise en charge dans la version actuelle du package utilisée
for row in src_wb[sheet_annexe_1].iter_rows(
    min_row=5, min_col=dico_cols[date_report], max_col=dico_cols[date_report]
):
    for cell in row:
        if has_formula(cell):
            code_produit, facteur_conversion = (
                src_wb[sheet_annexe_1].cell(cell.row, 1).value,
                src_wb[sheet_annexe_1].cell(cell.row, 8).value,
            )
            df = df_stock_prog_nat.loc[df_stock_prog_nat.Code_produit == int(code_produit)]
            if not df.empty:
                value = (
                    math.ceil(df.CONSO.sum() / int(facteur_conversion))
                    if not pd.isna(facteur_conversion) and facteur_conversion != 0
                    else 0
                )
                cell = src_wb[sheet_annexe_1].cell(
                    row=cell.row, column=dico_cols[date_report], value=value
                )
            else:
                cell = src_wb[sheet_annexe_1].cell(
                    row=cell.row, column=dico_cols[date_report], value=0
                )

del df

In [ ]:
# CMM échélonnée par mois
interface.clear_cache()
data_list = []
for start, row in enumerate(
    src_wb[sheet_annexe_1].iter_rows(
        min_row=4,
        min_col=pyxl.utils.column_index_from_string("BL"),
        max_col=pyxl.utils.column_index_from_string("CJ"),
    ),
    start=1,
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    if start == 1:
        new_data = []
        unnamed_counter = 65  # Colonne de début des CMM
        for value in data:
            if value is None:
                new_data.append(f"Unnamed: {unnamed_counter}")
                unnamed_counter += 2
            else:
                new_data.append(value)
        data = new_data
    data_list.append(data)
del new_data

In [ ]:
df_cmm = pd.concat(
    [df_annexe[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
)

In [ ]:
df_stock_track_cmm = df_cmm[[col for col in df_cmm.columns if "Unnamed" not in str(col)]]

df_stock_track_cmm = pd.melt(
    df_stock_track_cmm, id_vars="code_produit", var_name="date_report", value_name="cmm"
)  # .drop_duplicates()

df_stock_track_cmm["date_report"] = (
    df_stock_track_cmm["date_report"]
    .apply(lambda x: pd.to_datetime(str(x)[:10], format="%Y-%m-%d"))
    .astype("<M8[ns]")
)

In [ ]:
# Extraction des informations pour le mois courant
interface.clear_cache()
data_list = []
for row in src_wb[sheet_annexe_1].iter_rows(
    min_row=3,
    min_col=pyxl.utils.column_index_from_string("CW"),
    max_col=pyxl.utils.column_index_from_string("CZ"),
):
    data = []
    for cell in row:
        if gstf.utils.has_formula(cell):
            result = interface.calc_cell(cell.coordinate, sheet_annexe_1)
            data.append(result)
        else:
            data.append(cell.value)
    data_list.append(data)

In [ ]:
df_cmm_currenth_month = pd.concat(
    [df_annexe[["code_produit"]], pd.DataFrame(data_list[1:], columns=data_list[0])],
    axis=1,
).dropna(how="all")

df_cmm_currenth_month["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")
df_cmm_currenth_month.columns = df_cmm_currenth_month.columns.str.replace("\n", " ")
df_cmm_currenth_month.rename(
    columns={
        "Nbre de mois de considérés": "nbre_mois_consideres",
        "Consommations enregistrées sur les mois de considérés": "conso_mois_consideres",
        "CMM Calculée en fin du mois": "cmm_calculee",
        "COMMENTAIRE": "commentaire",
    },
    inplace=True,
)

df_cmm_currenth_month.head(2)

In [ ]:
assert (
    df_stock_track_cmm.merge(
        df_cmm_currenth_month, how="left", on=["code_produit", "date_report"]
    ).shape[0]
    == df_stock_track_cmm.shape[0]
)

df_stock_track_cmm = df_stock_track_cmm.merge(
    df_cmm_currenth_month, how="left", on=["code_produit", "date_report"]
)

In [ ]:
# CMM histo
cols_df_cmm = df_cmm.columns.to_list()

mapping = {
    col: cols_df_cmm[i - 1] for i, col in enumerate(cols_df_cmm) if "Unnamed" in str(col) and i > 0
}


df = pd.melt(
    df_cmm,
    id_vars=["code_produit"],
    value_vars=[col for col in df_cmm.columns if not pd.isna(col)],
    var_name="date_report",
)

df["date_report"] = df["date_report"].map(mapping)

df = df.loc[df.value.notna()]

df_stock_track_cmm_histo = (
    df_stock_track_cmm[["code_produit", "date_report", "cmm"]]
    .merge(df.drop(columns="value"), on=["code_produit", "date_report"])
    .rename(columns={"date_report": "date_report_prev"})
)

df_stock_track_cmm_histo["date_report"] = pd.to_datetime(date_report, format="%Y-%m-%d")

del df, mapping

df_stock_track_cmm_histo.head()

## 3. Recherche des modifications sur `Annexe 2 - Suivi des Stocks`

In [ ]:
%%time
sheet_annexe_2 = compute_indicators.utils.check_if_sheet_name_in_file(
    "Annexe 2 - Suivi des Stocks", sheetnames
)
interface.clear_cache()
data_list = []
columns_letter = []
for start, row in enumerate(
    src_wb[sheet_annexe_2].iter_rows(
        min_row=5,
        min_col=pyxl.utils.column_index_from_string("I"),
        max_col=pyxl.utils.column_index_from_string("AR"),
    ),
    start=1,
):
    data = []
    for cell in row:
        try:
            if gstf.has_formula(cell):
                result = interface.calc_cell(cell.coordinate, sheet_annexe_2)
                data.append(result)
            else:
                data.append(cell.value)
        except:
            continue
    data_list.append(data)

In [ ]:
# L'ancienne valeur de column letter permet d'identifier les valeurs suivantes
columns_letter = [
    "SDU_CENTRAL",
    "DMM_CENTRAL",
    "MSD_CENTRAL",
    "STATUT_CENTRAL",
    "CONSO_DECENTRALISE",
    "SDU_DECENTRALISE",
    "CMM_DECENTRALISE",
    "MSD_DECENTRALISE",
    "STATUT_DECENTRALISE",
    "nombre_de_site_en_rupture_annexe_2",
    "SDU_NATIONAL",
    "CMM_NATIONAL",
    "MSD_NATIONAL",
    "STATUT_NATIONAL",
    "Date de Péremption la plus proche (BRUTE)",
    "Date de Péremption la plus proche",
    "Quantité correspondante",
    "MSD correspondant",
    "Durée d'utilisation à la NPSP (mois)",
    "Nombre de jours restant avant l'expiration",
    "Moins de 6 mois (RED)",
    "Entre 6 et 12 mois (ORANGE)",
    "Plus de 12 mois (GREEN)",
    "Qtité attendue Annexe 2",
    "MSD attendu Annexe 2",
    "Qtité réceptionnés non en Stock",
    "MSD reçu",
    "Financement",
    "Date Probable de Livraison",
    "Date Effective de Livraison",
    "Statut",
    "Analyse du risque / Commentaires",
    "Diligences au niveau Central",
    "Diligences au niveau périphérique",
    "Responsable",
    "Dilig. Choisie",
]

In [ ]:
df_annexe = pd.concat([df_annexe, pd.DataFrame(data_list, columns=columns_letter)], axis=1)

df_annexe.fillna(np.nan, inplace=True)

del data_list, data

df_annexe.head(3)